In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

/home/iria/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/iria/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/iria/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/iria/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWa

Utils

In [2]:
def xavier_init(size):
    # Weight initializer
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


def plot(samples):
    # 3x3 plot for show generative net's output
    fig = plt.figure(figsize=(3, 3))
    gs = gridspec.GridSpec(3, 3)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

# Building networks

Discriminative net 

In [3]:
mnist_shape = 784
hidden_size = 256
output_shape = 10

input_D_ph = tf.placeholder(tf.float32, [None, mnist_shape], name='Real_X')

D_W1 = tf.Variable(xavier_init((mnist_shape, hidden_size)), name='D_W1')
D_b1 = tf.Variable(tf.zeros(hidden_size), name='D_b1')

D_W2 = tf.Variable(xavier_init((hidden_size, output_shape)), name='D_W2')
D_b2 = tf.Variable(tf.zeros(output_shape), name='D_b2')


def discriminative_network(x):
    layer1 = tf.matmul(x, D_W1) + D_b1
    layer1 = tf.nn.relu(layer1)
    
    layer2 = tf.matmul(layer1, D_W2) + D_b2
    layer2 = tf.nn.sigmoid(layer2)

    return layer2

Instructions for updating:
Colocations handled automatically by placer.


Generative net

In [4]:
z_dim = 300
input_G_ph = tf.placeholder(tf.float32, [None, z_dim], name='Fake_X')

G_W1 = tf.Variable(xavier_init((z_dim, hidden_size)), name='G_W1')
G_b1 = tf.Variable(tf.zeros(hidden_size), name='G_b1')

G_W2 = tf.Variable(xavier_init((hidden_size, mnist_shape)), name='G_W2')
G_b2 = tf.Variable(tf.zeros(mnist_shape), name='G_b2')


def generative_network(z):
    layer1 = tf.matmul(z, G_W1) + G_b1
    layer1 = tf.nn.relu(layer1)
    
    layer2 = tf.matmul(layer1, G_W2) + G_b2
    layer2 = tf.nn.sigmoid(layer2)

    return layer2


def noise_generator(shape):
    noise = np.random.uniform(-1., 1., size=shape)
    return noise

# Calculating net's outputs and losses

In [5]:
fake_input = generative_network(input_G_ph)

D_real = discriminative_network(input_D_ph)
D_fake = discriminative_network(fake_input)

D_real_loss = tf.reduce_mean(tf.log(D_real))
D_fake_loss = tf.reduce_mean(tf.log(1. - D_fake))
D_loss = -(D_real_loss + D_fake_loss)

G_loss = -tf.reduce_mean(tf.log(D_fake))

# Training

In [ ]:
lr = 0.0001
D_train = tf.train.AdamOptimizer(learning_rate=lr).minimize(D_loss, var_list=[D_W1, D_b1, D_W2, D_b2])
G_train = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss, var_list=[G_W1, G_b1, G_W2, G_b2])

In [ ]:
mnist_ex = input_data.read_data_sets("./data/", one_hot=True)

epoches = 1500
batch_size = 4096

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True
sess = tf.Session()#config=config)

sess.run(tf.initialize_all_variables())
with sess:
    for e in range(epoches):
        d_losses, g_losses = [], []
        
        if e % 5 == 0:
            noise = noise_generator((9, z_dim))
            samples = sess.run(generative_network(input_G_ph), feed_dict={input_G_ph: noise})

            fig = plot(samples)
            plt.savefig('./out/{}.png'.format(str(e).zfill(3)), bbox_inches='tight')
            plt.close(fig)
            
        
        for step in range(mnist_ex.train.num_examples // batch_size):
            batchX, _ = mnist_ex.train.next_batch(batch_size)
            noise = noise_generator((batch_size, z_dim))
            
            d_loss, _ = sess.run([D_loss, D_train], feed_dict={input_D_ph: batchX, input_G_ph: noise})
            g_loss, _ = sess.run([G_loss, G_train], feed_dict={input_G_ph: noise})
            
            d_losses.append(d_loss)
            g_losses.append(g_loss)
        
        print("Epoch %d   D loss: %.5f   G loss: %.5f" % (e, d_loss, g_loss))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0   D loss: 1.02128   G loss: 1.53794
Epoch 1   D loss: 0.79131   G loss: 1.96383
Epoch 2   D loss: 0.61004   G loss: 2.20527
Epoch 3   D loss: 0.46489   G loss: 2.38224
Epoch 4   D loss: 0.36062   G loss: 2.56504
Epoch 5   D loss: 0.283

Epoch 127   D loss: 0.58814   G loss: 1.35965
Epoch 128   D loss: 0.60085   G loss: 1.35409
Epoch 129   D loss: 0.60861   G loss: 1.32297
Epoch 130   D loss: 0.60178   G loss: 1.34032
Epoch 131   D loss: 0.58839   G loss: 1.35597
Epoch 132   D loss: 0.57371   G loss: 1.36265
Epoch 133   D loss: 0.57226   G loss: 1.36428
Epoch 134   D loss: 0.56793   G loss: 1.37715
Epoch 135   D loss: 0.56498   G loss: 1.38587
Epoch 136   D loss: 0.53931   G loss: 1.39953
Epoch 137   D loss: 0.53037   G loss: 1.42134
Epoch 138   D loss: 0.52380   G loss: 1.42655
Epoch 139   D loss: 0.51945   G loss: 1.41925
Epoch 140   D loss: 0.52661   G loss: 1.40658
Epoch 141   D loss: 0.51163   G loss: 1.43687
Epoch 142   D loss: 0.50241   G loss: 1.44359
Epoch 143   D loss: 0.49400   G loss: 1.45621
Epoch 144   D loss: 0.47534   G loss: 1.48568
Epoch 145   D loss: 0.47462   G loss: 1.51211
Epoch 146   D loss: 0.44245   G loss: 1.55463
Epoch 147   D loss: 0.45582   G loss: 1.55467
Epoch 148   D loss: 0.45094   G lo